- ## Advice_Identifier_From_Doctor's_Voice_Prescription

In [1]:
import spacy

In [2]:
from tqdm import tqdm
import spacy
import random
import json
import re

In [3]:
training_data = []
lines=[]
with open('MEDICAL_ADVICE.json', 'r',encoding="utf8") as f:
    lines = f.readlines()

for line in lines:
    data = json.loads(line)
    text = data['content']
    entities = []
    if(data['annotation'] is not None):
        for annotation in data['annotation']:
              if(annotation is not None):
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                    entities.append((point['start'], point['end'] + 1 ,label))

    if(entities != []):
        training_data.append((text, {"entities" : entities}))

In [4]:
len(training_data)

17

In [5]:
training_data[4:15]

[('Abhijit kaushal, your age is 29 yeras.According to your query you have the symptoms of Hoarseness,Persistent lumps,swollen glands.The problem i found that you are  suffering from Tetanus.The prescribed medicine you have to take are pyrantel,Tablet (chewable): 250 mg (as embonate orpamoate).Floss or use interdental brushes at least once a day to clean between your teeth.Change your toothbrush every 3–4 months.try Physical activity which promotes blood circulation.',
  {'entities': [(412, 466, 'ADVICE'),
    (372, 411, 'ADVICE'),
    (300, 371, 'ADVICE')]}),
 ('You are  Yashwant of age 30 years.According to your query you are the having the problems of Hoarseness,Persistent lumps,swollen glands.The diagnosis report shows that you have the symptoms of Fifth disease.Medicines albendazole Tablet (chewable): 400 mg,ivermectin Tablet (scored): 3 mg.take small frequent meals.You should take complete bed rest.',
  {'entities': [(314, 348, 'ADVICE'), (288, 313, 'ADVICE')]}),
 ('Hi Mr Avnish C

In [6]:
## Training a blank spacy english model.

nlp = spacy.blank('en')  # create blank Language class
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)

# add labels
for _, annotations in tqdm(training_data):
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(20):
        print("Statring iteration " + str(itn))
        random.shuffle(training_data)
        losses = {}
        for text, annotations in training_data:
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.2,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)

100%|██████████| 17/17 [00:00<00:00, 8743.49it/s]


Statring iteration 0
{'ner': 508.2927660606205}
Statring iteration 1
{'ner': 184.23398949815297}
Statring iteration 2
{'ner': 199.96196750970915}
Statring iteration 3
{'ner': 307.292838427423}
Statring iteration 4
{'ner': 112.19064099245365}
Statring iteration 5
{'ner': 106.69040318161258}
Statring iteration 6
{'ner': 42.67644469480392}
Statring iteration 7
{'ner': 30.105218777831194}
Statring iteration 8
{'ner': 45.471992532560805}
Statring iteration 9
{'ner': 64.45962611683512}
Statring iteration 10
{'ner': 23.757781450974637}
Statring iteration 11
{'ner': 48.021418095545066}
Statring iteration 12
{'ner': 31.646068363254514}
Statring iteration 13
{'ner': 35.42434349871715}
Statring iteration 14
{'ner': 29.082801850413862}
Statring iteration 15
{'ner': 11.538083528591999}
Statring iteration 16
{'ner': 32.24703219273429}
Statring iteration 17
{'ner': 50.01083937991246}
Statring iteration 18
{'ner': 44.967517761627825}
Statring iteration 19
{'ner': 19.894942703775815}


In [12]:
nlp.to_disk("medical_Advice_model")

In [17]:
import spacy
nlp4=spacy.load("medical_Advice_model")

In [42]:
doc=nlp4(open("./VKCCC/MR_10.txt").read())

In [43]:
print(doc)

Hi Miss Kumari Sneha, Your age is 43 years.According to your problems i have read you are having the problems of difficulty swallowing,Unusual vaginal bleeding.According to your report,There is a chance of disease Volvulus.You have to take the medicines HalfLytely and Bisacodyl: 15 mg , Acetaminophen: 520 mg;Haloperidol Decanoate 1 g (as sodium salt),Halothane :250 mg or 1 g (as pentahydrate) in vial.You should wash fruits and vegetables befor eating,You should avoid contact with people.Try to eat black grapes and not jumk foods.




In [44]:
for entity in doc.ents:
    print(entity.text)

You should wash fruits and vegetables befor eating
You should avoid contact with people
Try to eat black grapes and not jumk foods
